In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import Mean
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model

from skimage.transform import resize
from skimage.io import imread


In [8]:

# Define paths to the dataset files
train_input_path = "/Volumes/Personal/SDP/Brain tumor MRI and CT scan/data(processed)/train_input.npy"
train_output_path = "/Volumes/Personal/SDP/Brain tumor MRI and CT scan/data(processed)/train_output.npy"
val_input_path = "/Volumes/Personal/SDP/Brain tumor MRI and CT scan/data(processed)/val_input.npy"
val_output_path = "/Volumes/Personal/SDP/Brain tumor MRI and CT scan/data(processed)/val_output.npy"
test_input_path = "/Volumes/Personal/SDP/Brain tumor MRI and CT scan/data(processed)/test_input.npy"
test_output_path = "/Volumes/Personal/SDP/Brain tumor MRI and CT scan/data(processed)/test_output.npy"

# Load the data
train_input = np.load(train_input_path)
train_output = np.load(train_output_path)
val_input = np.load(val_input_path)
val_output = np.load(val_output_path)
test_input = np.load(test_input_path)
test_output = np.load(test_output_path)

# Print shapes to ensure correct loading
print("Train Input Shape:", train_input.shape)
print("Train Output Shape:", train_output.shape)
print("Validation Input Shape:", val_input.shape)
print("Validation Output Shape:", val_output.shape)


Train Input Shape: (570, 256, 256)
Train Output Shape: (570, 256, 256)
Validation Input Shape: (90, 256, 256)
Validation Output Shape: (90, 256, 256)


In [9]:
# Define preprocessing functions
def normalize_image(image):
    """Normalize image pixel values to range [0, 1]."""
    return image / 255.0

def preprocess_input(images):
    """Preprocess input images."""
    preprocessed_images = []
    for image in images:
        # Normalize image
        normalized_image = normalize_image(image)
        preprocessed_images.append(normalized_image)
    return np.array(preprocessed_images)

def preprocess_output(images):
    """Preprocess output images."""
    return preprocess_input(images)  # Assuming same preprocessing for output as input

# Preprocess the data
train_input_preprocessed = preprocess_input(train_input)
train_output_preprocessed = preprocess_output(train_output)
val_input_preprocessed = preprocess_input(val_input)
val_output_preprocessed = preprocess_output(val_output)


In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Dropout, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import Mean

# Define U-Net model with two input branches
def unet_model(input_shape):
    ct_input = Input(shape=input_shape)
    mri_input = Input(shape=input_shape)

    # Contracting Path for CT
    conv1_ct = Conv2D(64, 3, activation='relu', padding='same')(ct_input)
    conv1_ct = Conv2D(64, 3, activation='relu', padding='same')(conv1_ct)
    pool1_ct = MaxPooling2D(pool_size=(2, 2))(conv1_ct)

    # Contracting Path for MRI
    conv1_mri = Conv2D(64, 3, activation='relu', padding='same')(mri_input)
    conv1_mri = Conv2D(64, 3, activation='relu', padding='same')(conv1_mri)
    pool1_mri = MaxPooling2D(pool_size=(2, 2))(conv1_mri)

    # Concatenate the features from both paths
    merged = concatenate([pool1_ct, pool1_mri], axis=-1)

    # Continue with the rest of the U-Net architecture
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(merged)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)

    # Expansive Path
    up6 = Conv2D(512, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(drop5))
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = Conv2D(256, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = Conv2D(128, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = Conv2D(64, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv8))
    merge9 = concatenate([conv1_ct, conv1_mri, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs=[ct_input, mri_input], outputs=[conv10])

    return model

# Define model parameters
img_height = 256
img_width = 256
num_channels = 1
input_shape = (img_height, img_width, num_channels)

# Create U-Net model
model = unet_model(input_shape)

# Compile the model
model.compile(optimizer=Adam(), loss=MeanSquaredError(), metrics=[MeanSquaredError()])

# Print model summary
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_22 (Conv2D)  │ (None, 256, 256,  │        640 │ input_layer_2[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_24 (Conv2D)  │ (None, 256, 256,  │        640 │ input_layer_3[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_23 (Conv2D)  │ (None, 256, 256,  │     36,928 │ conv2d_22[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_25 (Conv2D)  │ (None, 256, 256,  │     36,928 │ conv2d_24[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_5     │ (None, 128, 128,  │          0 │ conv2d_23[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_6     │ (None, 128, 128,  │          0 │ conv2d_25[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 128, 128,  │          0 │ max_pooling2d_5[… │
│ (Concatenate)       │ 128)              │            │ max_pooling2d_6[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_26 (Conv2D)  │ (None, 128, 128,  │    147,584 │ concatenate_4[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_27 (Conv2D)  │ (None, 128, 128,  │    147,584 │ conv2d_26[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_7     │ (None, 64, 64,    │          0 │ conv2d_27[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_28 (Conv2D)  │ (None, 64, 64,    │    295,168 │ max_pooling2d_7[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_29 (Conv2D)  │ (None, 64, 64,    │    590,080 │ conv2d_28[0][0]   │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_8     │ (None, 32, 32,    │          0 │ conv2d_29[0][0]   │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_30 (Conv2D)  │ (None, 32, 32,    │  1,180,160 │ max_pooling2d_8[… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_31 (Conv2D)  │ (None, 32, 32,    │  2,359,808 │ conv2d_30[0][0] 

 Total params: 31,178,753 (118.94 MB)

 Trainable params: 31,178,753 (118.94 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Load the preprocessed dataset

train_input_ct = np.load("/Volumes/Personal/SDP/Brain tumor MRI and CT scan/data(processed)/train_input_ct.npy")
train_input_mri = np.load("/Volumes/Personal/SDP/Brain tumor MRI and CT scan/data(processed)/train_input_mri.npy")
train_output = np.load("/Volumes/Personal/SDP/Brain tumor MRI and CT scan/data(processed)/train_output.npy")

val_input_ct = np.load("/Volumes/Personal/SDP/Brain tumor MRI and CT scan/data(processed)/val_input_ct.npy")
val_input_mri = np.load("/Volumes/Personal/SDP/Brain tumor MRI and CT scan/data(processed)/val_input_mri.npy")
val_output = np.load("/Volumes/Personal/SDP/Brain tumor MRI and CT scan/data(processed)/val_output.npy")


FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/Personal/SDP/Brain tumor MRI and CT scan/data(processed)/train_input_ct.npy'